In [1]:
%matplotlib inline
from matplotlib import animation, rc
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,HBox
from IPython.display import display
import ipywidgets as widgets
from IPython.display import HTML
from math import sqrt,sin,pi
from numpy import empty
import matplotlib.pyplot as plt
import random
import time
import numba as nb
import numpy as np
"""
First we define our box with set parameters where:
x10: amplitude of our waves
side: side of our square or pond we are dropping the two pebbles in
points: number of points we want to carry our wave (number of medium points)
spacing: spacing between those points defined above
"""


"""
We define our widgets within our Jupyter Notebook,
we define 6 sliders where the first two are properties of our wave (wavelength and period),
second two are the coordinates of the first pebble (x1,y1) and the last two are the coordinates of the 
second pebble (x2,y2)

The interact_manual code calls the F function below with the parameters defined by the sliders. 
"""



side = 100.0          
points = 500      
spacing = side/points  



@nb.njit(nb.f8 (nb.f8, nb.f8, nb.f8),
        cache=True, fastmath=True, nogil=True)
def normalize(minimum, maximum, value):
    return (value-minimum)/(maximum-minimum)

def f2(frame,wavelength, pebbles, separation, speed):
    k = 2*pi/wavelength
    amp =1.0 #amplitude             
    spacing = side/points      
    minimum= -amp*pebbles
    maximum= amp*pebbles
    
    xi = empty([points,points],float)
    
    x_list = empty(pebbles, float)
    y_list = empty(pebbles, float)
    
    for i in range(pebbles):
        x = (side - ((pebbles - 1) * separation)) / 2 + (separation * i)
        y = side / 2
        
        x_list[i] = x
        y_list[i] = y
        
    calc_xi2(points, pebbles, amp, frame, speed, xi, x_list,y_list,k)
    im = plt.imshow(xi,origin="lower",extent=[0,side,0,side])
    return im

def update_plot(wavelength, separation, speed, pebbles):
    
    t0 = time.perf_counter() # time start
    frames=100
    fig = plt.figure(figsize=[5,5], dpi=100) 
    ax = plt.axes(xlim=(0, side), ylim=(0, side))
    plt.gray()
    
    ims = []
    for frame in range(frames):
        im=f2(frame,wavelength, pebbles, separation, speed)
        ims.append([im])
    
    plt.colorbar(im,orientation='horizontal')
    
    anim = animation.ArtistAnimation(fig, ims, blit=True, repeat=True)
    anim.save('Vibe Check.mp4')
    
    print("DONE!")
    t1 = time.perf_counter() # time end
    t = t1 - t0
    print("Time it took:",t,"!")

@nb.njit(nb.void(nb.i8, nb.i8, nb.f8, nb.i8, nb.f8, nb.f8[:,:], nb.f8[:],nb.f8[:], nb.f8),
        cache=True, fastmath=True, nogil=True)
def calc_xi2(points, pebbles, amp, frame, speed, xi, x_list,y_list,k):
    for i in range(points):
        y = spacing*i
        for j in range(points):
            x = spacing*j
            
            r = []
            for l in range(pebbles):
                x1 = x_list[l]
                y1 = y_list[l]
                
                r.append(sqrt((x-x1)**2+(y-y1)**2))
            
            total = 0
            for l in range(pebbles):
                total += amp * (sin(k*r[l] -(frame* speed)))
                
            xi[i,j] = normalize(-amp*pebbles, amp*pebbles, total)


widgets.interact_manual(update_plot,
                        wavelength=widgets.FloatSlider(min=0, max=10, value=5 ,step=.5, description="Wavelength"),
                        separation=widgets.FloatSlider(min=5, max=50, value=25, step=5, descritption="Separation"),
                        speed=widgets.FloatSlider(min=0,max=1,value=.25, step=.05, description="Speed"),
                        pebbles=widgets.Dropdown(options=[1,2,3,4,5], value=1, description= '# of Pebbles:', disabled=False))

interactive(children=(FloatSlider(value=5.0, description='Wavelength', max=10.0, step=0.5), FloatSlider(value=…

<function __main__.update_plot(wavelength, separation, speed, pebbles)>

In [2]:
from IPython.display import Video
Video("Vibe Check.mp4")